<a href="https://colab.research.google.com/github/sittidetw/dads5001_midterm_project/blob/main/EDA_HIV_Proactive_Screening.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Demographic and Spatial Factors to Enhance Proactive HIV Screening Efficiency

In [ ]:
!git clone https://github.com/sittidetw/dads5001_midterm_project.git

Cloning into 'dads5001_midterm_project'...
remote: Enumerating objects: 185, done.
remote: Counting objects: 100% (185/185), done.
remote: Compressing objects: 100% (180/180), done.
remote: Total 185 (delta 63), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (185/185), 2.06 MiB | 21.10 MiB/s, done.
Resolving deltas: 100% (63/63), done.


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Set styling
sns.set_theme(style="whitegrid")
plt.rcParams['font.sans-serif'] = ['Tahoma', 'Arial'] # For Thai language support if needed

# 1. Load the Data
file_path = '/content/dads5001_midterm_project/prep_data/hiv_pop_long.xlsx'
try:
    df = pd.read_excel(file_path)
    print("Data loaded successfully. Shape:", df.shape)
    display(df.head())
except FileNotFoundError:
    print(f"File not found at {file_path}. Please upload the file to colab.")


Data loaded successfully. Shape: (8470, 14)


,year,health_region,province_th,health_region_desc_th,region_th,province,health_region_desc,region,sex,population,test,positive,reach,prep
0,2021,1,น่าน,ภาคเหนือตอนบน (ล้านนา),ภาคเหนือ,Nan,Upper Northern (Lanna),North,female,all_kps,62,2,NaN,NaN
1,2021,1,น่าน,ภาคเหนือตอนบน (ล้านนา),ภาคเหนือ,Nan,Upper Northern (Lanna),North,female,fsw,0,0,NaN,NaN
2,2021,1,น่าน,ภาคเหนือตอนบน (ล้านนา),ภาคเหนือ,Nan,Upper Northern (Lanna),North,female,migrant,0,0,NaN,NaN
3,2021,1,น่าน,ภาคเหนือตอนบน (ล้านนา),ภาคเหนือ,Nan,Upper Northern (Lanna),North,female,msm,1,0,NaN,NaN
4,2021,1,น่าน,ภาคเหนือตอนบน (ล้านนา),ภาคเหนือ,Nan,Upper Northern (Lanna),North,female,msw,0,0,NaN,NaN


---
## ข้อ 1: ภาพรวมและช่องว่างของข้อมูล (Overall Positivity Rate & KPs vs Other)
ตรวจสอบอัตราการตรวจพบเชื้อในภาพรวม และเปรียบเทียบระหว่างกลุ่ม Key Populations (KPs) และกลุ่มคนทั่วไป (Other)

*(หมายเหตุ: ปรับชื่อคอลัมน์ `population_group`, `test_results`, หรือ `hiv_status` ให้ตรงกับชุดข้อมูลจริง)*

In [ ]:
# สมมติว่าข้อมูลเป็นระดับบุคคล (Row-level) มีคอลัมน์ Population_Group และ Test_Result ('Positive' / 'Negative')
# หากเป็นตารางสรุป (Aggregated) ที่มีคอลัมน์ Total_Tested และ Positive_Cases ให้ปรับสูตรการคำนวณ

if 'Test_Result' in df.columns and 'Population_Group' in df.columns:
    # คำนวณภาพรวม
    overall_positivity = (df['Test_Result'] == 'Positive').mean() * 100
    print(f"Overall Positivity Rate: {overall_positivity:.2f}%")

    # เปรียบเทียบ KPs vs Other
    positivity_by_group = df.groupby('Population_Group')['Test_Result'].apply(lambda x: (x == 'Positive').mean() * 100).reset_index()
    positivity_by_group.columns = ['Population_Group', 'Positivity_Rate_Percent']

    display(positivity_by_group)

    # กราฟแท่ง
    plt.figure(figsize=(8, 5))
    sns.barplot(data=positivity_by_group, x='Population_Group', y='Positivity_Rate_Percent', palette='Set2')
    plt.title('Positivity Rate by Population Group (KPs vs Other)')
    plt.ylabel('Positivity Rate (%)')
    plt.show()
else:
    print("กรุณาเปลี่ยนชื่อคอลัมน์ในโค้ดส่วนนี้ให้ตรงกับข้อมูลจริง (เช่น 'Population_Group', 'Test_Result')")


---
## ข้อ 2: การระบุกลุ่มประชากรแฝง (Hidden KPs)
เปรียบเทียบการกระจายตัวของผู้ติดเชื้อแยกตามเพศ ดูความผิดปกติในกลุ่ม "Other Female" เทียบกับ "KPs Male"

*(หมายเหตุ: ต้องมีคอลัมน์ `Gender` และแยกกลุ่มประชากร)*

In [ ]:
# เลือกเฉพาะผู้ที่มีผลตรวจเป็น Positive
if 'Test_Result' in df.columns and 'Gender' in df.columns:
    positive_df = df[df['Test_Result'] == 'Positive']

    # สร้างกลุ่มเปรียบเทียบ (Group + Gender)
    positive_df['Subgroup'] = positive_df['Population_Group'] + "_" + positive_df['Gender']

    # สัดส่วนของผู้ติดเชื้อแต่ละกลุ่มย่อย
    subgroup_dist = positive_df['Subgroup'].value_counts(normalize=True).reset_index()
    subgroup_dist.columns = ['Subgroup', 'Proportion']
    subgroup_dist['Proportion'] *= 100

    plt.figure(figsize=(10, 6))
    sns.barplot(data=subgroup_dist, x='Subgroup', y='Proportion', palette='muted')
    plt.title('Distribution of Positive Cases by Group and Gender')
    plt.ylabel('Proportion among Positives (%)')
    plt.xticks(rotation=45)
    plt.axhline(y=subgroup_dist[subgroup_dist['Subgroup'] == 'Other_Female']['Proportion'].values[0] if 'Other_Female' in subgroup_dist['Subgroup'].values else 0, color='r', linestyle='--', label="Other Female Level")
    plt.legend()
    plt.show()
else:
    print("กรุณาตั้งชื่อคอลัมน์ Gender, Test_Result ให้ตรงกับ Dataset")


---
## ข้อ 3: การเจาะลึกประสิทธิภาพการตรวจ (Yield in Other Female)
ดูว่า Other Female มีอัตราการเจอเชื้อ (Yield) ต่ำหรือสูง หากสูงแปลว่าควรปรับปรุงการคัดกรองระบุตัวตนใหม่หรือไม่

In [ ]:
if 'Test_Result' in df.columns and 'Gender' in df.columns:
    # คำนวณ Yield แยกรายกลุ่มและเพศ
    yield_df = df.groupby(['Population_Group', 'Gender'])['Test_Result'].apply(lambda x: (x == 'Positive').mean() * 100).reset_index()
    yield_df.columns = ['Population_Group', 'Gender', 'Yield_Percent']

    display(yield_df)

    plt.figure(figsize=(8, 6))
    sns.barplot(data=yield_df, x='Population_Group', y='Yield_Percent', hue='Gender', palette='pastel')
    plt.title('Screening Yield by Base Population and Gender')
    plt.ylabel('Yield / Positivity Rate (%)')
    plt.show()

    print("ข้อสังเกต: หากแท่งของ Other Female มีค่าเปอร์เซ็นต์ที่เข้าใกล้ KPs แสดงว่า Screening Criteria ของกลุ่ม Other อาจจะยังมีช่องโหว่ในการแยกแยะแฝง")


---
## ข้อ 4: การวิเคราะห์เชิงพื้นที่ (High Effort, Low Yield vs Low Effort, High Yield)
วิเคราะห์ 2 แกนหลักคือจำนวนการตรวจ (Effort) และอัตราการตรวจเจอ (Yield) ในแต่ละพื้นที่

In [ ]:
# สมมติว่ามีคอลัมน์ Area หรือ Province
if 'Area' in df.columns:
    spatial_df = df.groupby('Area').agg(
        Total_Tested=('Test_Result', 'count'), # Effort
        Positive_Cases=('Test_Result', lambda x: (x == 'Positive').sum())
    ).reset_index()

    spatial_df['Yield_Percent'] = (spatial_df['Positive_Cases'] / spatial_df['Total_Tested']) * 100

    effort_mean = spatial_df['Total_Tested'].median()
    yield_mean = spatial_df['Yield_Percent'].median()

    plt.figure(figsize=(10, 8))
    sns.scatterplot(data=spatial_df, x='Total_Tested', y='Yield_Percent', s=100)

    # สร้าง Quadrant Lines
    plt.axvline(x=effort_mean, color='gray', linestyle='--')
    plt.axhline(y=yield_mean, color='gray', linestyle='--')

    plt.title('Spatial Analysis of Action Case Finding (Effort vs Yield)')
    plt.xlabel('Effort (Total Tests)')
    plt.ylabel('Yield (Positivity Rate %)')

    # เพิ่ม Labels ให้กับบางพื้นที่
    for i, row in spatial_df.iterrows():
        plt.text(row['Total_Tested'] + 10, row['Yield_Percent'], row['Area'], fontsize=9)

    plt.show()

    print("Quadrant Analysis:")
    print(" - บนซ้าย (Low Effort, High Yield): พื้นที่เป้าหมายที่ควรเพิ่มทรัพยากรการตรวจ")
    print(" - ล่างขวา (High Effort, Low Yield): พื้นที่ที่ใช้ทรัพยากรเยอะแต่เจอน้อย อาจต้องปรับกลยุทธ์ Risk Identification ให้แม่นยำขึ้น")
else:
    print("กรุณาปรับชื่อคอลัมน์ 'Area' หรือ 'Location' เพื่อใช้ในกราฟนี้")
